In [2]:
import pandas as pd
from sqlalchemy import create_engine, inspect

# Store CSV into DataFrame

In [3]:
#Load in energy data
csv_file_1 = "resources/energy_census_and_economic_data_US_2010-2014.csv"
energy_df = pd.read_csv(csv_file_1)
energy_df.head()

,StateCodes,State,Region,Division,Coast,Great Lakes,TotalC2010,TotalC2011,TotalC2012,TotalC2013,...,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014
0,AL,Alabama,3.0,6.0,1.0,0.0,1931522,1905207,1879716,1919365,...,1.165832,1.157861,-0.020443,-0.168414,0.396416,0.420102,1.011941,1.001333,1.562247,1.577963
1,AK,Alaska,4.0,9.0,1.0,0.0,653221,653637,649341,621107,...,3.203618,2.869760,-1.175137,-1.949571,-3.789313,-13.754494,0.948185,1.835376,-0.585695,-10.884734
2,AZ,Arizona,4.0,8.0,0.0,0.0,1383531,1424944,1395839,1414383,...,1.090035,1.091283,1.341472,-0.420875,-0.580562,-1.313050,2.317801,0.621971,0.509473,-0.221767
3,AR,Arkansas,3.0,7.0,0.0,0.0,1120632,1122544,1067642,1096438,...,2.141877,2.129805,1.369514,5.131282,3.910476,6.280636,3.336628,7.155212,6.052353,8.410441
4,CA,California,4.0,9.0,1.0,0.0,7760629,7777115,7564063,7665241,...,4.207353,4.177389,-1.162079,-1.173951,-1.341226,-0.830982,2.761377,2.772770,2.866127,3.346406


In [4]:
#Load in minimum wage data
csv_file_2 = "resources/minimum_wage_data.csv"
minimum_wage_df = pd.read_csv(csv_file_2,encoding='ISO-8859-1')
minimum_wage_df.head()

,Year,State,State.Minimum.Wage,State.Minimum.Wage.2020.Dollars,Federal.Minimum.Wage,Federal.Minimum.Wage.2020.Dollars,Effective.Minimum.Wage,Effective.Minimum.Wage.2020.Dollars,CPI.Average,Department.Of.Labor.Uncleaned.Data,Department.Of.Labor.Cleaned.Low.Value,Department.Of.Labor.Cleaned.Low.Value.2020.Dollars,Department.Of.Labor.Cleaned.High.Value,Department.Of.Labor.Cleaned.High.Value.2020.Dollars,Footnote
0,1968,Alabama,0.00000,0.00,1.15,8.55,1.15,8.55,34.8,...,0.00000,0.00,0.00000,0.00,NaN
1,1968,Alaska,2.10000,15.61,1.15,8.55,2.10,15.61,34.8,2.1,2.10000,15.61,2.10000,15.61,NaN
2,1968,Arizona,0.46800,3.48,1.15,8.55,1.15,8.55,34.8,18.72 - 26.40/wk(b),0.46800,3.48,0.66000,4.91,(b)
3,1968,Arkansas,0.15625,1.16,1.15,8.55,1.15,8.55,34.8,1.25/day(b),0.15625,1.16,0.15625,1.16,(b)
4,1968,California,1.65000,12.26,1.15,8.55,1.65,12.26,34.8,1.65(b),1.65000,12.26,1.65000,12.26,(b)


In [37]:
#Cleaning Energy Consumption Data
year = []
state = []
energy_consumption = []
gdp = []


for index, row in energy_df.iterrows():
    year.append('2010')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2010'][index])
    gdp.append(energy_df['GDP2010'][index])
    

for index, row in energy_df.iterrows():
    year.append('2011')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2011'][index])
    gdp.append(energy_df['GDP2011'][index])
    
for index, row in energy_df.iterrows():
    year.append('2012')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2012'][index])
    gdp.append(energy_df['GDP2012'][index])
    
for index, row in energy_df.iterrows():
    year.append('2013')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2013'][index])
    gdp.append(energy_df['GDP2013'][index])
    
for index, row in energy_df.iterrows():
    year.append('2014')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2014'][index])
    gdp.append(energy_df['GDP2014'][index])

    

In [38]:
energy_df_cleaned = pd.DataFrame({
#     'year_state': year_state,
    'year': year,
    'state': state,
    'total_energy': energy_consumption ,
    'gdp': gdp
})

energy_df_cleaned

,year,state,total_energy,gdp
0,2010,Alabama,1931522,176220.75
1,2010,Alaska,653221,54220.00
2,2010,Arizona,1383531,247333.00
3,2010,Arkansas,1120632,104928.75
4,2010,California,7760629,1960935.00
...,...,...,...,...
255,2014,West Virginia,752942,74432.75
256,2014,Wisconsin,1868867,293341.25
257,2014,Wyoming,535612,40875.50
258,2014,District of Columbia,178929,116539.25


In [47]:
#Cleaning Minimum Wage Data
# minimum_wage_df_cleaned = minimum_wage_df[['Year', 'State', 'State.Minimum.Wage', 'State.Minimum.Wage.2020.Dollars']].copy()
# minimum_wage_df_cleaned.head()
reduced_min_wage_data_df = minimum_wage_df[['Year', 'State', 'State.Minimum.Wage', 'CPI.Average']].copy()
renamed_min_wage_data_df = reduced_min_wage_data_df.rename(columns={'Year':'year', 'State':'state', 'State.Minimum.Wage': 'state_minimum_wage', 'CPI.Average':'cpi_average'})

cleaned_min_wage_data_df = renamed_min_wage_data_df.loc[  (renamed_min_wage_data_df['year'] > 2009) & (renamed_min_wage_data_df['year'] < 2015)]
cleaned_min_wage_data_df

,year,state,state_minimum_wage,cpi_average
2268,2010,Alabama,0.00,218.056
2269,2010,Alaska,7.75,218.056
2270,2010,Arizona,7.25,218.056
2271,2010,Arkansas,6.25,218.056
2272,2010,California,8.00,218.056
...,...,...,...,...
2533,2014,Virginia,7.25,236.736
2534,2014,Washington,9.32,236.736
2535,2014,West Virginia,7.25,236.736
2536,2014,Wisconsin,7.25,236.736
